# Polycube robustness

In [1]:
import analyse_output as utils
import polycubes
import altair as alt
import numpy as np
import pandas as pd

Enumerate all genotypes one point mutation away

In [2]:
def enumerateMutations(hexRule, maxColor=8, maxCubes=8, dim=3):
    emptyCube = "000000000000";
    orientations = range(4)
    samedir = [1,1,2,0]
    colors = range(-maxColor, maxColor+1)
    # Pad with empty cubes
    while len(hexRule) < maxCubes * len(emptyCube):
        hexRule += emptyCube;
        
    mutations = []
    rule = utils.parseHexRule(hexRule)
    for i, cube in enumerate(rule):
        for j, face in enumerate(cube):
            for color in colors:
                if color != face['color']:
                    newRule = utils.parseHexRule(hexRule);
                    newRule[i][j]['color'] = color
                    mutations.append(utils.ruleToHex(newRule))
            if dim == 3:
                for orientation in orientations:
                    if orientation != face['orientation']:
                        newRule = utils.parseHexRule(hexRule);
                        newRule[i][j]['orientation'] = orientation
                        mutations.append(utils.ruleToHex(newRule))
    return mutations;

## Genotype robustness

Calculate the fraction of mutational neigbours that produce the same phenotype:

In [3]:
def calcGenotypeRobustness(hexRule, maxColor=8, maxCubes=8, dim=3):
    mutations = enumerateMutations(hexRule, maxColor, maxCubes, dim);
    nEqual = sum(polycubes.checkEquality(hexRule, mutant) for mutant in mutations)
    return nEqual / len(mutations)

Test for a small example square:

In [4]:
calcGenotypeRobustness('040087000000', 3, 2)

0.7592592592592593

 If we allow a larger phenotype space the robustness will be higher, since there are more room for neutral mutations:

In [5]:
data = []
for maxCol in range(1,10):
    for maxCubes in range(1,10):
        robustness = calcGenotypeRobustness('040087000000', maxCol, maxCubes)
        data.append({'maxCol': maxCol, 'maxCubes': maxCubes, 'robustness': robustness})
alt.Chart(pd.DataFrame(data=data)).mark_rect().encode(
    x='maxCol:O',
    y='maxCubes:O',
    color='robustness'
)

alt.Chart(...)